# Semantic Web KEN3140 - Lab 4
## RDFS Reasoning (and intro to OWL Axioms) in Python 

### The Semantics of Axioms about Properties

|                           IF                             |               THEN              |        Rule          |
|:--------------------------------------------------------:|:-------------------------------:|:-------------------------:|
| `T(?p, rdfs:domain, ?c) T(?x, ?p, ?y)`                   | `T(?x, rdf:type, ?c)`           | **Domain Inference** <tr></tr> |
| `T(?p, rdfs:range, ?c)  T(?x, ?p, ?y)`                   | `T(?y, rdf:type, ?c)`           | **Range Inference** <tr></tr> |
| `T(?p1 , rdfs:subPropertyOf, ?p2 ) T(?x, ?p1 , ?y)`      | `T(?x, ?p2 , ?y)`               | **Subproperty Inheritance** <tr></tr> |


### The Semantics of Class Axioms
|                           IF                             |               THEN              |        Rule        |
|:--------------------------------------------------------:|:-------------------------------:|:-----------------------:|
| `T(?c1, rdfs:subClassOf, ?c2) T(?x, rdf:type, ?c1)`      | `T(?x, rdf:type, ?c2)`           | **Type Inheritance** <tr></tr> |


### The Semantics of Schema Vocabulary

|                                     IF                                     |                        THEN                         |      Rule     |
|:---------------------------------------------------------------------------:|:---------------------------------------------------:|:------------------:|
| `T(?c1 , rdfs:subClassOf, ?c2 )  T(?c2 , rdfs:subClassOf, ?c3 )`            | `T(?c1 , rdfs:subClassOf, ?c3 )`                    | **Transitivity of Subclass** <tr></tr> |
| `T(?c1 , rdfs:subClassOf, ?c2 )  T(?c2 , rdfs:subClassOf, ?c1 )`            | `T(?c1 , owl:equivalentClass, ?c2 )`                | **Class Equivalence** <tr></tr> |
| `T(?p1 , rdfs:subPropertyOf, ?p2 )  T(?p2 , rdfs:subPropertyOf, ?p3 )`      | `T(?p1 , rdfs:subPropertyOf, ?p3 )`                 | **Transitivity of Subproperty** <tr></tr> |
| `T(?p1 , rdfs:subPropertyOf, ?p2 )  T(?p2 , rdfs:subPropertyOf, ?p1 )`      | `T(?p1 , owl:equivalentProperty, ?p2 )`             | **Property Equivalence** <tr></tr> |
| `T(?p , rdfs:domain, ?c1 )  T(?c1 , rdfs:subClassOf, ?c2 )`                 | `T(?p , rdfs:domain, ?c2 )`                         | **Domain Propagation** <tr></tr> |
| `T(?p2 , rdfs:domain, ?c )  T(?p1 , rdfs:subPropertyOf, ?p2 )`              | `T(?p1 , rdfs:domain, ?c )`                         | **Domain Inheritance** <tr></tr> |
| `T(?p , rdfs:range, ?c1 )  T(?c1 , rdfs:subClassOf, ?c2 )`                  | `T(?p , rdfs:range, ?c2 )`                          | **Range Propagation** <tr></tr> |
| `T(?p2 , rdfs:range, ?c )  T(?p1 , rdfs:subPropertyOf, ?p2 )`               | `T(?p1 , rdfs:range, ?c )`                          | **Range Inheritance** <tr></tr> |



### Install RDFLib (if not done already) and OWL-RL

In [2]:
import sys
!{sys.executable} -m pip install rdflib
!{sys.executable} -m pip install git+https://github.com/RDFLib/OWL-RL.git

  Cloning https://github.com/RDFLib/OWL-RL.git to c:\users\timur\appdata\local\temp\pip-req-build-0_qk7us8
  Resolved https://github.com/RDFLib/OWL-RL.git to commit 8e1bdbb322c656fcb7f6745447d740eec40fc18e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/RDFLib/OWL-RL.git 'C:\Users\Timur\AppData\Local\Temp\pip-req-build-0_qk7us8'


### Task 1: Recreate the graph below using RDFLib
```nt
@prefix ex: <http://example.com/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:kevin a ex:Teenager ;
    ex:hasAge 17.5 .

ex:angela ex:hasSister ex:julie ;
    ex:isMotherOf ex:kevin .

ex:julie ex:hasBrother ex:john .
```

In [4]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, OWL, XSD, URIRef
from owlrl import DeductiveClosure, OWLRL_Semantics

# Namespaces
EX = Namespace("http://example.com/")
graph = Graph()
graph.bind("ex", EX)
graph.bind("rdfs", RDFS)
graph.bind("rdf", RDF)
graph.bind("xsd", XSD)

# Classes
graph.add((EX.Teenager, RDF.type, RDFS.Class))
graph.add((EX.Person, RDF.type, RDFS.Class))

# Properties
graph.add((EX.hasAge, RDF.type, RDF.Property))
graph.add((EX.hasAge, RDFS.domain, EX.Teenager))
graph.add((EX.hasAge, RDFS.range, XSD.float))

graph.add((EX.hasSister, RDF.type, RDF.Property))
graph.add((EX.hasSister, RDFS.domain, EX.Person))
graph.add((EX.hasSister, RDFS.range, EX.Person))

graph.add((EX.hasBrother, RDF.type, RDF.Property))
graph.add((EX.hasBrother, RDFS.domain, EX.Person))
graph.add((EX.hasBrother, RDFS.range, EX.Person))

graph.add((EX.isMotherOf, RDF.type, RDF.Property))
graph.add((EX.isMotherOf, RDFS.domain, EX.Person))
graph.add((EX.isMotherOf, RDFS.range, EX.Person))

# Individuals
graph.add((EX.kevin, RDF.type, EX.Teenager))
graph.add((EX.kevin, EX.hasAge, Literal(17.5, datatype=XSD.float)))

graph.add((EX.angela, RDF.type, EX.Person))
graph.add((EX.angela, EX.hasSister, EX.julie))
graph.add((EX.angela, EX.isMotherOf, EX.kevin))

graph.add((EX.julie, RDF.type, EX.Person))
graph.add((EX.julie, EX.hasBrother, EX.john))

graph.add((EX.john, RDF.type, EX.Person))

# Run reasoner (OWL RL)
DeductiveClosure(OWLRL_Semantics).expand(graph)

# Print inferred triples
for s, p, o in graph:
    print(s, p, o)


http://www.w3.org/2001/XMLSchema#NCName http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://example.com/julie http://www.w3.org/2002/07/owl#sameAs http://example.com/julie
http://www.w3.org/2001/XMLSchema#unsignedLong http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2002/07/owl#sameAs http://www.w3.org/2002/07/owl#sameAs http://www.w3.org/2002/07/owl#sameAs
http://www.w3.org/2001/XMLSchema#unsignedLong http://www.w3.org/2002/07/owl#sameAs http://www.w3.org/2001/XMLSchema#unsignedLong
http://www.w3.org/2001/XMLSchema#NCName http://www.w3.org/2002/07/owl#sameAs http://www.w3.org/2001/XMLSchema#NCName
http://www.w3.org/2001/XMLSchema#dateTimeStamp http://www.w3.org/2002/07/owl#sameAs http://www.w3.org/2001/XMLSchema#dateTimeStamp
http://example.com/hasBrother http://www.w3.org/2002/07/owl#sameAs http://example.com/hasBrother
http://www.w3.org/2001/XMLSchema#Name http://www.w3.or

In [5]:
# Functions - leave as is
def print_related_triples(graph, subjects):
    """Print all triples in the graph related to the given subjects."""
    for subject in subjects:
        print(f"Triples with subject {subject}:")
        for s, p, o in graph:
            if s == URIRef(subject) and  p != URIRef("http://www.w3.org/2002/07/owl#sameAs"):
                print(f"{s}, {p}, {o}")
        print("\n")

def copy_graph(g):
    g_ = Graph()
    for s, p, o in g:
        g_.add((s, p, o))
    return g_

subjects = [
    "http://example.com/kevin",
    "http://example.com/angela",
    "http://example.com/john",
    "http://example.com/julie"
]

In [6]:
# Print triples from graph as is
print_related_triples(graph, subjects)

Triples with subject http://example.com/kevin:
http://example.com/kevin, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://example.com/Person
http://example.com/kevin, http://example.com/hasAge, 17.5
http://example.com/kevin, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://example.com/Teenager


Triples with subject http://example.com/angela:
http://example.com/angela, http://example.com/hasSister, http://example.com/julie
http://example.com/angela, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://example.com/Person
http://example.com/angela, http://example.com/isMotherOf, http://example.com/kevin


Triples with subject http://example.com/john:
http://example.com/john, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://example.com/Person


Triples with subject http://example.com/julie:
http://example.com/julie, http://example.com/hasBrother, http://example.com/john
http://example.com/julie, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://example.com/Pe

### Type Inheritance and (sub)Property Inheritance

In [7]:
# Adding a few schema triples using RDFS.subClassOf, RDFS.subPropertyOf
g2 = copy_graph(graph)
g2.add((EX.isMotherOf, RDFS.subPropertyOf, EX.isParentOf))
g2.add((EX.hasSister, RDFS.subPropertyOf, EX.hasSibling))
g2.add((EX.hasBrother, RDFS.subPropertyOf, EX.hasSibling))
g2.add((EX.Teenager, RDFS.subClassOf, EX.Adolescent))
g2.add((EX.Adolescent, RDFS.subClassOf, EX.Person))

# Build the closure
DeductiveClosure(OWLRL_Extension).expand(g2)

# Print triples from
print_related_triples(g2, subjects)

Triples with subject http://example.com/kevin:
http://example.com/kevin, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://www.w3.org/2002/07/owl#Thing
http://example.com/kevin, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://www.w3.org/2000/01/rdf-schema#Resource
http://example.com/kevin, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://example.com/Person
http://example.com/kevin, http://example.com/hasAge, 17.5
http://example.com/kevin, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://example.com/Teenager
http://example.com/kevin, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://example.com/Adolescent


Triples with subject http://example.com/angela:
http://example.com/angela, http://example.com/hasSister, http://example.com/julie
http://example.com/angela, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://www.w3.org/2002/07/owl#Thing
http://example.com/angela, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://www.w3.org/2000/01/rdf-sche

### Let's define the range and domain of the properties using  ```RDFS.domain```, and ```RDFS.range``` and look at **Domain Inference** and **Range Inference**
1. ```hasSister```
   ```nt
    ex:hasSister a owl:ObjectProperty ;
        rdfs:domain ex:Person ;
        rdfs:range ex:Female .
   ```
2. ```hasBrother```
   ```nt
    ex:hasBrother a owl:ObjectProperty ;
        rdfs:domain ex:Person ;
        rdfs:range ex:Male .

### Task 2: 
1. Copy graph `g2` to a new graph `g3`
2. Add the above triples using RDFLib to `g3`
3. Run ```DeductiveClosure``` to infer new triples.
4. Print the triples using `print_related_triples`

In [ ]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL, XSD, Literal
from owlrl import DeductiveClosure, OWLRL_Semantics

EX = Namespace("http://example.com/")

g3 = Graph()
g3.bind("ex", EX); g3.bind("rdfs", RDFS); g3.bind("owl", OWL); g3.bind("rdf", RDF); g3.bind("xsd", XSD)
for t in g2:            # если g2 уже есть в памяти
    g3.add(t)

g3.add((EX.Person,  RDF.type, RDFS.Class))
g3.add((EX.Female,  RDF.type, RDFS.Class))
g3.add((EX.Male,    RDF.type, RDFS.Class))

g3.add((EX.hasSister,  RDF.type, OWL.ObjectProperty))
g3.add((EX.hasSister,  RDFS.domain, EX.Person))   
g3.add((EX.hasSister,  RDFS.range,  EX.Female))   

g3.add((EX.hasBrother, RDF.type, OWL.ObjectProperty))
g3.add((EX.hasBrother, RDFS.domain, EX.Person))   
g3.add((EX.hasBrother, RDFS.range,  EX.Male))     

DeductiveClosure(OWLRL_Semantics).expand(g3)

def print_related_triples(g, node):
    for s,p,o in g.triples((node, None, None)):
        print("S:", s, " P:", p, " O:", o)
    for s,p,o in g.triples((None, None, node)):
        print("S:", s, " P:", p, " O:", o)

print("=== ANGELA ===")
print_related_triples(g3, EX.angela)

print("\n=== JULIE ===")
print_related_triples(g3, EX.julie)

print("\n=== JOHN ===")
print_related_triples(g3, EX.john)


### Moving beyond RDFS
We will now look at axioms beyond those built with `rdfs:subClassOf`, `rdfs:subPropertyOf`, `rdfs:domain`, and `rdfs:range`.  
1. Define `hasSibling` as a symmetric relation

In [ ]:
g4 = copy_graph(g3)
g4.add((EX.hasSibling, RDF.type, OWL.SymmetricProperty))
# Build the closure
DeductiveClosure(OWLRL_Extension).expand(g4)
# Print triples from
print_related_triples(g4, subjects)

2. Define `hasSibling` as a transitive relation

In [ ]:
g5 = copy_graph(g4)
g5.add((EX.hasSibling, RDF.type, OWL.TransitiveProperty))
# Build the closure
DeductiveClosure(OWLRL_Extension).expand(g5)
# Print triples from
print_related_triples(g5, subjects)

```nt
In the above inference, notice the degenerate case of considering full-siblinghood as a transitive property. i.e. you are your own sibling. 
```

### Extending inference using `OWL.propertyChainAxiom`. Defining >1 hop relations: 
1. `hasAunt`
```nt
ex:hasAunt a owl:ObjectProperty ;
    rdfs:domain ex:Person ;
    rdfs:range ex:Female ;
    owl:propertyChainAxiom ( ex:hasParent ex:hasSister ) .
```

In [ ]:
def create_rdf_list(graph, items):
    list_bnode = BNode()
    collection = Collection(graph, list_bnode, items)
    return list_bnode

g6 = copy_graph(g5)

g6.add((EX.hasParent, OWL.inverseOf, EX.isParentOf))
g6.add((EX.hasAunt, RDF.type, OWL.ObjectProperty))
g6.add((EX.hasAunt, RDFS.domain, EX.Person))
g6.add((EX.hasAunt, RDFS.range, EX.Female))
g6.add((EX.hasAunt, OWL.propertyChainAxiom, create_rdf_list(g6, [EX.hasParent, EX.hasSister])))

DeductiveClosure(OWLRL_Extension).expand(g6)

# Print triples from
print_related_triples(g5, subjects)
print("--"*25)
print_related_triples(g6, subjects)

### Task 3: 
Use `OWL.propertyChainAxiom` from above to define `hasUncle` relation and infer that `Kevin` `hasUncle` `John`
1. `hasUncle`
```nt
ex:hasUncle a owl:ObjectProperty ;
    rdfs:domain ex:Person ;
    rdfs:range ex:Male ;
    owl:propertyChainAxiom ( ex:hasParent ex:hasBrother ) .
```

In [ ]:
## copy g6 to g7
g7 = copy_graph(g6)
## Add triples to g7

# Print triples from
print_related_triples(g6, subjects)
print("--"*25)
print_related_triples(g7, subjects)

Did the inclusion of `propertyChainAxiom` for `hasUncle` result in the intended output. If no, why? Hint: Look for the `hasBrother` relation between `Angela` and `John`. 

In [ ]:
## copy g6 to g7
g7 = copy_graph(g6)
## Add triples 

# Print triples from
print_related_triples(g6, subjects)
print("--"*25)
print_related_triples(g7, subjects)

Finally, does the above definition of `hasBrother` or `hasSister` always result in inferring all relevant relations? No. For example it will not infer `Julie` `hasSister` `Angela` or `John` `hasSister` `Angela` because `X` `hasSister` `Angela` is not a triple that already exists. If it existed for either `Julie` or `John` it could have been inferred for the other. Moreover, `hasSister` is niether `symmetric` nor `transitive`. Ultimately, we need extra information, i.e. gender to ascertain this relation --> `hasSibling(A,B) AND Female(B) => hasSister(A,B)`. 